In [2]:
import wave
import numpy as np
import matplotlib.pyplot as plt
import stumpy
import os
from moviepy.editor import *

In [1]:
# Used to separate audio from the video. The audio will later be used for synchronization

# left = mp.VideoFileClip(r"left.mp4")

# left.audio.write_audiofile(r"left.wav")

# right = mp.VideoFileClip(r"right.mp4")

# right.audio.write_audiofile(r"right.wav")

In [ ]:
print(os.getcwd())
left_wav = wave.open("left.wav", "rb")
right_wav = wave.open("right.wav", "rb")

In [ ]:
left_freq = left_wav.getframerate()
right_freq = right_wav.getframerate()

left_samples = left_wav.getnframes()
right_samples = right_wav.getnframes()

left_signal = left_wav.readframes(left_samples)
right_signal = right_wav.readframes(right_samples)

print(len(left_signal))
print(len(right_signal))
print(left_samples/left_freq)
print(right_samples/right_freq)

left_audio = left_samples/left_freq
right_audio = right_samples/right_freq

In [ ]:
left_signal_array = np.frombuffer(left_signal, dtype=np.int16)
right_signal_array = np.frombuffer(right_signal, dtype=np.int16)

print(np.shape(left_signal_array))
print(np.shape(right_signal_array))

In [ ]:
times_left = np.linspace(0, left_samples/left_freq, num=left_samples)
times_right = np.linspace(0, right_samples/right_freq, num=right_samples)

# Use right channel for left camera and left channel for right camera
left_channel = left_signal_array[1::2]
right_channel = right_signal_array[0::2]


# fft_left = np.fft.fft(left_channel)
# fft_right = np.fft.fft(right_channel)

In [ ]:

plt.figure(figsize=(15, 5))
plt.plot(times_left, left_channel)
plt.title('Left Camera')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, left_audio)
plt.show()

In [ ]:

plt.figure(figsize=(15, 5))
plt.plot(times_right, right_channel)
plt.title('Right Camera')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, right_audio)
plt.show()

In [ ]:

# plt.figure(figsize=(15, 5))
# plt.plot(times_left, fft_left)
# plt.title('Left Camera')
# plt.ylabel('FFT Signal Value')
# plt.xlabel('Time (s)')
# plt.xlim(0, left_audio)
# plt.show()

In [ ]:

# plt.figure(figsize=(15, 5))
# plt.plot(times_right, fft_right)
# plt.title('Right Camera')
# plt.ylabel('Signal Value')
# plt.xlabel('Time (s)')
# plt.xlim(0, right_audio)
# plt.show()

In [ ]:
left_spike = left_channel.argmax()
right_spike = right_channel.argmax()

left_spike_start = left_spike-10000
left_spike_end = left_spike+10000

right_spike_start = right_spike-10000
right_spike_end = right_spike+10000

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(times_left, left_channel)
plt.plot(times_left[left_spike_start:left_spike_end], left_channel[left_spike_start:left_spike_end], color="r")
plt.title('Left Camera')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, left_audio)
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(times_right, right_channel)
plt.plot(times_right[right_spike-10000:right_spike+10000], right_channel[right_spike-10000:right_spike+10000], color="r")

plt.title('Right Camera')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, right_audio)
plt.show()

In [ ]:
m = 10000
left_subarray = left_channel[left_spike_start:left_spike_end]
right_subarray = right_channel[right_spike_start:right_spike_end]

In [ ]:
matrix_profile = stumpy.stump(T_A=left_subarray.astype(np.float64), m=m, T_B=right_subarray.astype(np.float64), ignore_trivial=False)

In [ ]:
plt.plot(matrix_profile[:,0])
plt.title('Matrix Profile')
plt.ylabel('Value')
plt.xlabel('Subsequence Number')

In [ ]:
left_motif_index = matrix_profile[:,0].argmin()
right_motif_index = matrix_profile[left_motif_index,1] + right_spike_start
left_motif_index += left_spike_start
print(left_motif_index)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(times_left, left_channel)
plt.plot(times_left[left_motif_index:left_motif_index+m], left_channel[left_motif_index:left_motif_index+m], color="red")
plt.title('Left Camera')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, left_audio)
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(times_right, right_channel)
plt.plot(times_right[right_motif_index:right_motif_index+m], right_channel[right_motif_index:right_motif_index+m], color="red")
plt.title('Right Camera')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')
plt.xlim(0, right_audio)
plt.show()

In [ ]:
plt.plot(times_right[right_motif_index:right_motif_index+m], right_channel[right_motif_index:right_motif_index+m])
plt.title('Right Camera Most Similar Subsequence')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')

In [ ]:
plt.plot(times_left[left_motif_index:left_motif_index+m], left_channel[left_motif_index:left_motif_index+m])
plt.title('Left Camera Most Similar Subsequence')
plt.ylabel('Signal Value')
plt.xlabel('Time (s)')

In [ ]:
print(times_left[left_motif_index])
print(times_left[-1])
print(times_right[right_motif_index])
print(times_right[-1])

In [ ]:
# infile = 'left.mp4'
# outfile = 'left_test.ts'
# subprocess.run([r'C:\Program Files\ffmpeg\bin\ffmpeg.exe', '-i', infile, outfile])

In [ ]:
# infile = 'left_test.ts'
# outfile = 'left_test.mp4'
# subprocess.run([r'C:\Program Files\ffmpeg\bin\ffmpeg.exe', '-i', infile, outfile])

In [ ]:
# ffmpeg_extract_subclip(filename="left_test.ts", t1=times_left[left_motif_index]*1000, t2=times_left[-1]*1000, targetname="test.ts")
# #ffmpeg_extract_subclip("right.mp4", times_right[right_motif_index], times_right[-1], targetname="right_aligned.mp4")

In [ ]:
# infile = 'test.ts'
# outfile = 'test.mp4'
# subprocess.run([r'C:\Program Files\ffmpeg\bin\ffmpeg.exe', '-i', infile, outfile])

In [ ]:
# # THIS IS BROKEN FOR SOME REASON. CORRUPT VIDEO
# test = VideoFileClip("left.mp4")
# test = test.subclip(0,5)

# test.to_videofile("test.mp4", codec="libx264", temp_audiofile='temp-audio.m4a', remove_temp=True, audio_codec='aac')

In [ ]:
test = VideoFileClip("left.mp4")
test = test.subclip(times_left[left_motif_index],times_left[-1])

test.ipython_display(filename="left_trimmed.mp4", width=360)
os.rename("__temp__.mp4", "left_trimmed.mp4")

In [ ]:
test = VideoFileClip("right.mp4")
test = test.subclip(times_right[right_motif_index],times_right[-1])

test.ipython_display(filename="right_trimmed.mp4", width=360)
os.rename("__temp__.mp4", "right_trimmed.mp4")

In [ ]:
# TODO: Code to put videos side by side for comparison. 

# left_vid = VideoFileClip("left.mp4")
# left_vid = left_vid.subclip(times_left[left_motif_index],times_left[-1])

# right_vid = VideoFileClip("right.mp4")
# right_vid = right_vid.subclip(times_right[right_motif_index],times_right[-1])

# combined = clips_array([[left_vid, right_vid]])
# combined.ipython_display(width=360)
# os.rename("__temp__.mp4", "combined_trimmed.mp4")